In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [2]:
from movielens_data_new import MovieLens
import torch as th
import numpy as np
import dgl

dataset = MovieLens("ml-1m")

/Users/pratikaher/DGL/graph-rec/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting processing ml-1m ...


/Users/pratikaher/DGL/GRecSy/movielens_data_new.py:253: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movie_info = pd.read_csv(file_path, sep='::', header=None,


......
All rating edges : 1000209
	All train rating edges : 900188
		Train rating edges : 810169
		Valid rating edges : 90019
	Test rating edges  : 100021
Total user number = 6040, movie number = 3706
Feature dim: 
user: torch.Size([6040, 23])
movie: torch.Size([3706, 319])
Train enc graph: 	#user:6040	#movie:3706	#pairs:810169
Valid enc graph: 	#user:6040	#movie:3706	#pairs:90019
Test enc graph: 	#user:6040	#movie:3706	#pairs:100021


In [99]:
dataset.all_rating_info.head()

,user_id,movie_id,rating,timestamp,year,month,date,nearest_holiday,nearest_holiday_days
0,1,1193,5,2000-12-31 14:12:40,2000,12,2000-12-31,Christmas Day,6
1,1,661,3,2000-12-31 14:35:09,2000,12,2000-12-31,Christmas Day,6
2,1,914,3,2000-12-31 14:32:48,2000,12,2000-12-31,Christmas Day,6
3,1,3408,4,2000-12-31 14:04:35,2000,12,2000-12-31,Christmas Day,6
4,1,2355,5,2001-01-06 15:38:11,2001,1,2001-01-06,New Year's Day,5


In [24]:
from datetime import datetime

In [25]:
df_rating = dataset.all_rating_info

In [26]:
df_rating['timestamp'] = df_rating['timestamp'].apply(datetime.fromtimestamp)
df_rating['year'] = df_rating['timestamp'].dt.year
df_rating['month'] = df_rating['timestamp'].dt.month
df_rating['date'] = df_rating['timestamp'].dt.date

In [34]:
df_rating

,user_id,movie_id,rating,timestamp,year,month,date
0,1,1193,5,2000-12-31 14:12:40,2000,12,2000-12-31
1,1,661,3,2000-12-31 14:35:09,2000,12,2000-12-31
2,1,914,3,2000-12-31 14:32:48,2000,12,2000-12-31
3,1,3408,4,2000-12-31 14:04:35,2000,12,2000-12-31
4,1,2355,5,2001-01-06 15:38:11,2001,1,2001-01-06
...,...,...,...,...,...,...,...
1000204,6040,1091,1,2000-04-25 19:35:41,2000,4,2000-04-25
1000205,6040,1094,5,2000-04-25 16:21:27,2000,4,2000-04-25
1000206,6040,562,5,2000-04-25 16:19:06,2000,4,2000-04-25
1000207,6040,1096,4,2000-04-25 19:20:48,2000,4,2000-04-25


In [30]:
import holidays
us_holidays = holidays.US()

In [38]:
holiday_map = {}
L = []
for (date, holiday_name) in holidays.US(years = 2000).items():
    L.append((date, holiday_name))
holiday_map[2000] = L[:]
L = []
for (date, holiday_name) in holidays.US(years = 2001).items():
    L.append((date, holiday_name))
holiday_map[2001] = L[:]
L = []
for (date, holiday_name) in holidays.US(years = 2002).items():
    L.append((date, holiday_name))
holiday_map[2002] = L[:]
l = []
for (date, holiday_name) in holidays.US(years = 2002).items():
    L.append((date, holiday_name))
holiday_map[2003] = L[:]


In [87]:
def get_next_holiday(year, current_date):
    
    holidays_that_year = holiday_map[year]
    min_difference = float('inf')
    closest_holiday = ''
    
    for (holiday_date, holiday_name) in holidays_that_year:
        
        current_difference = abs((current_date - holiday_date).days)
        
        if current_difference < min_difference:
            
            closest_holiday = holiday_name
            min_difference = current_difference
    
    return closest_holiday
    

In [78]:
def get_next_holiday_date(year, current_date):
    
    holidays_that_year = holiday_map[year]
    min_difference = float('inf')
    
    for (holiday_date, holiday_name) in holidays_that_year:
        
        current_difference = abs((current_date - holiday_date).days)
        min_difference = min(min_difference, current_difference)
    
    return min_difference
    

In [88]:
df_rating['nearest_holiday'] = df_rating.apply(lambda row : get_next_holiday(row['year'], row['date']), axis = 1)

In [79]:
df_rating['nearest_holiday_days'] = df_rating.apply(lambda row : get_next_holiday_date(row['year'], row['date']), axis = 1)

In [89]:
df_rating.head()

,user_id,movie_id,rating,timestamp,year,month,date,nearest_holiday,nearest_holiday_days
0,1,1193,5,2000-12-31 14:12:40,2000,12,2000-12-31,Christmas Day,6
1,1,661,3,2000-12-31 14:35:09,2000,12,2000-12-31,Christmas Day,6
2,1,914,3,2000-12-31 14:32:48,2000,12,2000-12-31,Christmas Day,6
3,1,3408,4,2000-12-31 14:04:35,2000,12,2000-12-31,Christmas Day,6
4,1,2355,5,2001-01-06 15:38:11,2001,1,2001-01-06,New Year's Day,5


In [3]:
# dataset.train_enc_graph.nodes['user'].data['features'] = dataset.user_feature
# dataset.train_enc_graph.nodes['movie'].data['features'] = dataset.movie_feature

# train_rating_info = th.FloatTensor(dataset.train_rating_info['rating'].values.astype(np.float32))
# dataset.train_enc_graph.edges['rates'].data['rating'] =  train_rating_info
# dataset.train_enc_graph.edges['rev-rates'].data['rating'] =  train_rating_info

In [4]:
# dataset.valid_enc_graph.nodes['user'].data['features'] = dataset.user_feature
# dataset.valid_enc_graph.nodes['movie'].data['features'] = dataset.movie_feature

# valid_rating_info = th.FloatTensor(dataset.valid_rating_info['rating'].values.astype(np.float32))
# dataset.valid_enc_graph.edges['rates'].data['rating'] =  valid_rating_info
# dataset.valid_enc_graph.edges['rev-rates'].data['rating'] =  valid_rating_info

In [5]:
# dataset.test_enc_graph.nodes['user'].data['features'] = dataset.user_feature
# dataset.test_enc_graph.nodes['movie'].data['features'] = dataset.movie_feature

# test_rating_info = th.FloatTensor(dataset.test_rating_info['rating'].values.astype(np.float32))
# dataset.test_enc_graph.edges['rates'].data['rating'] =  test_rating_info
# dataset.test_enc_graph.edges['rev-rates'].data['rating'] =  test_rating_info

In [6]:
# dataset.train_enc_graph.edges['rates'].data['rating']
# dataset.train_enc_graph.edges['rates']
# dataset.train_enc_graph.edges(etype='rates')

In [101]:
train_enc_graph.nodes['movie'].data['node_ID']

NodeSpace(data={'features': tensor([[-0.2074,  0.0357, -0.3190,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2781, -0.0447,  0.0315,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0071, -0.4135, -0.1930,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 1.0057,  0.2101,  0.2003,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1855, -0.0969, -0.1628,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1218,  0.3169, -0.3073,  ...,  1.0000,  0.0000,  0.0000]])})

In [7]:
train_enc_graph = dataset.train_enc_graph
valid_enc_graph = dataset.valid_enc_graph

In [8]:
from grecsy_hetero.model import MPNNConvModel

params = {'hidden_dim' : 128, 'out_dim' : 64 }

dim_dict = {'user': train_enc_graph.nodes['user'].data['features'].shape[1],
            'movie': train_enc_graph.nodes['movie'].data['features'].shape[1],
            'out': params['out_dim'],
            'hidden': params['hidden_dim']}

In [9]:
train_eids_dict = {}
valid_eids_dict = {}

eids = np.arange(train_enc_graph.number_of_edges(etype='rates'))
eids = np.random.permutation(eids)

test_size = int(len(eids) * 0.1)
valid_size = int(len(eids) * 0.1)
train_size = len(eids) - test_size - valid_size

In [10]:
for e in train_enc_graph.etypes:
    train_eids_dict[e] = eids[:train_size]
    valid_eids_dict[e] = eids[train_size:train_size+valid_size]

In [11]:
sampler = dgl.dataloading.MultiLayerNeighborSampler([15, 10, 5])
train_dataloader = dgl.dataloading.EdgeDataLoader(
        train_enc_graph, train_eids_dict, 
        sampler, negative_sampler=dgl.dataloading.negative_sampler.Uniform(5), 
        shuffle=True, drop_last=False,
        batch_size=16
        )

/Users/pratikaher/DGL/graph-rec/venv/lib/python3.8/site-packages/dgl/dataloading/dataloader.py:968: DGLWarning: EdgeDataLoader directly taking a BlockSampler will be deprecated and it will not support feature prefetching. Please use dgl.dataloading.as_edge_prediction_sampler to wrap it.
  dgl_warning(


In [12]:
valid_dataloader = dgl.dataloading.EdgeDataLoader(
        valid_enc_graph, valid_eids_dict, 
        sampler, negative_sampler=dgl.dataloading.negative_sampler.Uniform(5), 
        shuffle=True, drop_last=False,
        batch_size=16
        )

In [13]:
model = MPNNConvModel(train_enc_graph, dim_dict)
optimizer = th.optim.Adam(model.parameters(), lr=0.0001,weight_decay=0)


In [14]:
from grecsy_hetero.model import compute_loss

In [15]:
def validation_single_epoch(valid_dataloader):
    
    model.eval()
    with torch.no_grad():
        total_val_loss = 0

        for _, pos_g, neg_g, blocks in valid_dataloader:

            optimizer.zero_grad()

            input_features = blocks[0].srcdata['features']

            _, pos_score, neg_score = model(blocks,
                                    input_features,
                                    pos_g,
                                    neg_g)

            # print(pos_score, neg_score)
            loss = compute_loss(pos_score, neg_score)
            total_val_loss += loss.item()
    
    return total_val_loss

In [16]:
def train_single_epoch(train_dataloader):
    
    total_loss = 0

    for _, pos_g, neg_g, blocks in train_dataloader:
        
        optimizer.zero_grad()

        input_features = blocks[0].srcdata['features']
        
        _, pos_score, neg_score = model(blocks,
                                        input_features,
                                        pos_g,
                                        neg_g)
        
#         print(pos_score, neg_score)

        loss = compute_loss(pos_score, neg_score)

#         optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    return total_loss
    

In [17]:
epochs = 10

In [92]:
for i in range(epochs):
    
    print(f"Epoch {i}")
    
    training_loss = train_single_epoch(train_dataloader)
    
    print(f"Training Loss : {training_loss}")
    
#     validation_loss = validation_single_epoch(valid_dataloader)
    
#     print(f"Validation Loss : {validation_loss}")
    

Epoch 0


RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [ ]:
a = th.empty([0])

In [ ]:
b = th.Tensor(1)

In [ ]:
a

In [ ]:
a = th.cat((a, b), 0)

In [ ]:
a